<a id="top"></a>
# WOMTOOL generated json template file get config.txt file values
## Alice in WDLand
<img src="images/HookahCaterpillerAliceInWDLand.jpg" width="200" height="300"> <br>


## WOMTOOL template 
    * Keys are decimal-point separated, call stack variable names.
    * Leftmost Key part is stack bottom (call-start).
    * Rightmost Key part is variable name used in Config file (HICKUP).
    * Values are wdl types defined in the WDL specification.
    
## Config.txt convention
```bash
variable_key_name="variable value"
variable_array_key_name=["variable value"]
```
    * Plain text file with unquoted string keys, equals sign and quoted string variable value.
    * arrays are indicated by brackets.
    * Note: config file keys may allow AMBIGUOUS json assignment.
****
[test data generation cells](#generate_test_data) <br>
****
[read json file](#read_json_file) <br>
[read config file](#read_config_file) <br>
[put config in json](#put_config_in_json) <br>
[write filled in json](#write_filled_in_json) <br>

In [46]:
import os
import yaml
import json
from collections import OrderedDict, defaultdict, Counter


def configure_json_dict(json_dict, config_dict):
    """ Usage: configured_dict, json_missing_dict, config_used_dict = configure_json_dict(json_dict, config_dict)
    
    Args:
        json_dict:          python dict from json template file
        config_dict:        python dict from config.txt file intended to fill in the json template
    Returns:
        configured_dict:    json dict keys: config dict values
        json_missing_dict:  keys: value (= types)  for json dict keys not found in config dict
        config_used_dict:   the config file key-value pairs used
    """
    configured_dict = defaultdict()
    json_missing_dict = defaultdict()
    config_used_dict = defaultdict()
    # empty dictionary of lists -- missing config values -- json_key: json_template_value
    
    keys_d = get_json_keys_config_dict(json_dict)
    # dictionary of json key -- rightmost member: json key (all.members.vars)
    
    json_keys_counter = Counter(json_dict.keys())
    # checks to be sure that all keys are filled with some value from the config file
    
    for k, v in config_dict.items():
        # put the config dict values in the json file full-key
        if k in keys_d:
            config_used_dict[k] = v
            # design decision to keep config used dict in usable format - even if ambiguous here
            for var_name in keys_d[k]:
                configured_dict[var_name] = v.strip('"')
                json_keys_counter[var_name] += 1
                
    for k, v in json_keys_counter.items():
        # counter starts at one ergo json key is missing if v is 1
        if v < 2:
            json_missing_dict[k] = json_dict[k]
                
    return configured_dict, json_missing_dict, config_used_dict


def get_json_keys_config_dict(json_dict):
    """ Usage:    keys_dict = get_json_keys_config_dict(json_dict) 
    
    Args:
        json_dict:      json template file as python dict
        
    Returns:
        keys_dict:      key is righmost member of json key
                        value is json key
    """
    keys_dict = defaultdict(list)
    for k, v in json_dict.items():
        k_list = k.split('.')
        keys_dict[k_list[-1]].append(k)
    
    return keys_dict


def get_config_file_dict(configfile_fullpath):
    """ Usage:   config_file_dict = get_config_file_dict(configfile_fullpath)
    
    User must insure plain text encoding and the format dictated by WOMTOOLS - template.json 
    key="value in a string form" 
    key="124.7654"
    key="0"
    key="true"
    key=""
    key=[[["value1", "value2"]]]
    
    Args:
        configfile_fullpath:    full path to formatted plain text file 
                                
    Returns:
        config_file_dict:       python dictionary of key-value pairs 
                                (suitable for json file insertion)
    """
    
    pairs_list = []
    
    with open(configfile_fullpath, 'r') as fh:
        lines = fh.readlines()
        
    for line in lines:
        l = line.strip().split("=")
        if len(l) > 0 and len(l[0]) > 0:
            lefty = l[0].strip()
            if len(lefty) == 0:
                continue

            if not lefty[0] == "#" and len(l) > 1 and len(l[1]) > 0:
                righty = l[1].strip()
                if len(righty) == 0:
                    righty = ' '
                        
                if len(lefty) > 0:
                    pairs_list.append((lefty, righty))

    if len(pairs_list) > 0:
        config_file_dict = OrderedDict(pairs_list)
    else:
        config_file_dict = {}

    return config_file_dict


def get_json_file_dict(data_fullfilename):
    """ Usage: json_dict = get_json_file_dict(data_fullfilename)
    
    Args:
        data_fullfilename: json or yaml format full path filename
                                quoted strings or not (if consistant) 
                                -but no lines start with tab characters
    Returns:
        json_dict:               python dictionary of name - value parameters.
    """
    with open(data_fullfilename, 'r') as fh:
        json_dict = yaml.load(fh)

    return json_dict


<a id="generate_test_data"></a>
## Generate Test Data files
    * json file has one variable not found in config
    * config file has variable not found in json
[Top](#top) <br>

In [2]:
%%writefile jjalltheway.json
{
    "wf0.task0.var0": "Boolean",
    "wf0.task0.var1": "Int",
    "wf0.task0.var2": "String",
    "wf0.task0.var3": "File",
    "wf0.task0.var4": "Array[File]",
    "wf0.task0.var5": "Array[Array[File]]",
    "wf0.var6": "Array[Array[Array[File]]]",
    "wf0.var7": "Array[Array[Array[File]]]"
}

Overwriting jjalltheway.json


In [3]:
%%writefile conf.txt
var0="true"
var1="true"
var2="true"
var3="fileA"
var4=["fileA","fileB"]
var5=[["fileA","fileB"],["fileC","fileD"]]
var6=[[["fileA","fileB"],["fileC","fileD"]],[["fileE","fileF"],["fileG","fileH"]]]
number9='quacks like a duck'


Overwriting conf.txt


<a id="read_json_file"></a> <br>
### Python function that reads WOMTOOL generated json template.
[json module in python3 standard library](https://docs.python.org/3/library/json.html) <br>
```python
json_dict = get_json_file_dict(data_fullfilename)
```
[Top](#top) <br>

In [47]:
# demonstrate get_run_file_dict on json with quotes:
TestTask_dir = os.getcwd()
TestTask_jason_file = 'jjalltheway.json'
json_fullfilename = os.path.join(TestTask_dir, TestTask_jason_file)

if os.path.isfile(json_fullfilename):
    json_template_dict = get_json_file_dict(json_fullfilename)
    print('{0} variables found\n'.format(len(json_template_dict)))
    if len(json_template_dict) > 0:
        for k, v in json_template_dict.items():
            print('%30s: %s'%(k,v))
else:
    print(json_fullfilename, '\nNot Found')


8 variables found

                wf0.task0.var0: Boolean
                wf0.task0.var1: Int
                wf0.task0.var2: String
                wf0.task0.var3: File
                wf0.task0.var4: Array[File]
                wf0.task0.var5: Array[Array[File]]
                      wf0.var6: Array[Array[Array[File]]]
                      wf0.var7: Array[Array[Array[File]]]


<a id="read_config_file"></a> <br>
## python function that reads (special.txt, yaml, json) config files.
```python
config_file_dict = get_config_file_dict(configfile_fullpath)
```
[Top](#top) <br>   

In [48]:
# demonstrate get_run_file_dict on json with quotes:
TestTask_dir = os.getcwd()
TestTask_jason_file = 'conf.txt'
json_fullfilename = os.path.join(TestTask_dir, TestTask_jason_file)

if os.path.isfile(json_fullfilename):
    CONFIG_txt_dict = get_config_file_dict(json_fullfilename)
    print('{0} variables found\n'.format(len(CONFIG_txt_dict)))
    if len(CONFIG_txt_dict) > 0:
        for k, v in CONFIG_txt_dict.items():
            print('%30s: %s'%(k,v))
else:
    print(json_fullfilename, '\nNot Found')


8 variables found

                          var0: "true"
                          var1: "true"
                          var2: "true"
                          var3: "fileA"
                          var4: ["fileA","fileB"]
                          var5: [["fileA","fileB"],["fileC","fileD"]]
                          var6: [[["fileA","fileB"],["fileC","fileD"]],[["fileE","fileF"],["fileG","fileH"]]]
                       number9: 'quacks like a duck'


<a id="put_config_in_json"></a> <br>
## python function puts the config dict o dicts into wombat variables dict.
```python
configured_dict, data_not_dict = configure_json_dict(json_dict, config_dict)
```
[Top](#top) <br>

In [49]:
configured_dict, json_missing_dict, config_used_dict = configure_json_dict(json_dict=json_template_dict,
                                                                           config_dict=CONFIG_txt_dict)

print('\nConfigured dictionary\n')
for k, v in configured_dict.items():
    print('%30s: %s'%(k,v))
    
print('\n\nMissing data\n')
for k, v in json_missing_dict.items():
    print('%30s: %s'%(k,v))
    
print('\n\nInserted data\n')
for k, v in config_used_dict.items():
    print('%30s: %s'%(k,v))


Configured dictionary

                wf0.task0.var0: true
                wf0.task0.var1: true
                wf0.task0.var2: true
                wf0.task0.var3: fileA
                wf0.task0.var4: ["fileA","fileB"]
                wf0.task0.var5: [["fileA","fileB"],["fileC","fileD"]]
                      wf0.var6: [[["fileA","fileB"],["fileC","fileD"]],[["fileE","fileF"],["fileG","fileH"]]]


Missing data

                      wf0.var7: Array[Array[Array[File]]]


Inserted data

                          var0: "true"
                          var1: "true"
                          var2: "true"
                          var3: "fileA"
                          var4: ["fileA","fileB"]
                          var5: [["fileA","fileB"],["fileC","fileD"]]
                          var6: [[["fileA","fileB"],["fileC","fileD"]],[["fileE","fileF"],["fileG","fileH"]]]


<a id="write_filled_in_json"></a> <br>
## python function that takes all the above, checks and writes the wombat_data_dict

```python
    # write the json file with python standard library
    with open(json_test_full_filename, 'w') as outfile_handle:
        json.dump(json_dict, outfile_handle, indent="    ")
        
    return json_test_full_filename
```
[Top](#top) <br>


# Development State = pending details for write by types in write function
    * filled in dictionary, template dictionary -- both are needed as input
        * S.T each may be written by type special.
        * because Arrays are special if String or File.
        
```python
def write_special_wombat(json_dict, json_template_dict, filename_prefix=None):
    
    # iterate through json_dict string-format each key-value IAW its type in the json_template_dict
    
    # open file handle an write the string literally
```

In [51]:
def write_special_wombat(json_dict, json_template_dict, filename_prefix='test'):
    
    filename_suffix = 'template.json'
    if len(filename_prefix) < 1:
        full_filename = 'test.' + filename_suffix
    else:
        full_filename = filename_prefix.strip('.') + filename_suffix
        
    # iterate through json_dict string-format each key-value IAW its type in the json_template_dict
    S = '{\n'
    for json_key, json_value in json_dict.items():
        S += '    "' + json_key + '":'
        if "Array" in json_template_dict[json_key]:
            S += ' ' + json_value + ',\n'
        else:
            S += ' "' + json_value + '",\n'
            
    out_string = S[:-2] + '\n}\n'
    # open file handle an write the string literally
    with open(full_filename, 'w') as fh:
        fh.writelines(out_string)
        
    return full_filename

ug_file = write_special_wombat(json_dict=configured_dict, 
                               json_template_dict=json_template_dict, 
                               filename_prefix='test')
print(ug_file)

testtemplate.json


In [52]:
with open(ug_file, 'r') as fh:
    lines = fh.readlines()
    
for line in lines:
    if '{' in line or '}' in line:
        print(line.strip())
    else:
        print('    ',line.strip())

{
     "wf0.task0.var0": "true",
     "wf0.task0.var1": "true",
     "wf0.task0.var2": "true",
     "wf0.task0.var3": "fileA",
     "wf0.task0.var4": ["fileA","fileB"],
     "wf0.task0.var5": [["fileA","fileB"],["fileC","fileD"]],
     "wf0.var6": [[["fileA","fileB"],["fileC","fileD"]],[["fileE","fileF"],["fileG","fileH"]]]
}


In [20]:
with open('test.FilledIn.json', 'r') as fh:
    lines = fh.readlines()
    
for line in lines:
    if '{' in line or '}' in line:
        print(line.strip())
    else:
        print('    ',line.strip())

{
     "wf0.task0.var0": "true",
     "wf0.task0.var1": "true",
     "wf0.task0.var2": "true",
     "wf0.task0.var3": "fileA",
     "wf0.task0.var4": "[fileA,fileB]",
     "wf0.task0.var5": "[[fileA,fileB],[fileC,fileD]]",
     "wf0.var6": "[[[fileA,fileB],[fileC,fileD]],[[fileE,fileF],[fileG,fileH]]]",
     "wf0.var7": "Array[Array[Array[File]]]"
}


In [33]:
for k, v in json_template_dict.items():
    print(k,v)

wf0.task0.var0 Boolean
wf0.task0.var1 Int
wf0.task0.var2 String
wf0.task0.var3 File
wf0.task0.var4 Array[File]
wf0.task0.var5 Array[Array[File]]
wf0.var6 Array[Array[Array[File]]]
wf0.var7 Array[Array[Array[File]]]


In [ ]:
def write_filled_in_json_dict(json_dict, json_test_full_filename):
    
    # write the json file with python standard library
    with open(json_test_full_filename, 'w') as outfile_handle:
        json.dump(json_dict, outfile_handle, indent="    ")
        
    return json_test_full_filename


In [ ]:
with open('test.FilledIn.json', 'r') as fh:
    lines = fh.readlines()
    
for line in lines:
    if '{' in line or '}' in line:
        print(line.strip())
    else:
        print('    ',line.strip())